<a href="https://colab.research.google.com/github/namami-13/Twitter-Sentiment-Analyzer/blob/master/Namami_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import zipfile

In [ ]:
import pandas as pd
import os

# Verify if file exists and check size
file_path = 'training.1600000.processed.noemoticon.csv'
if os.path.exists(file_path):
    file_size = os.path.getsize(file_path)
    print(f"File exists. Size: {file_size/1024/1024:.2f} MB")
    # Expected size should be around 80MB for the full dataset
else:
    print("File not found. Downloading...")
    !wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    !unzip trainingandtestdata.zip

File not found. Downloading...
--2025-09-01 16:24:19--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2025-09-01 16:24:19--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  48.7MB/s    in 1.6s    

2025-09-01 16:24:21 (48.7 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv

In [ ]:
import pandas as pd
import os

# Verify if file exists and check size
file_path = 'training.1600000.processed.noemoticon.csv'
if os.path.exists(file_path):
    file_size = os.path.getsize(file_path)
    print(f"File exists. Size: {file_size/1024/1024:.2f} MB")
    # Expected size should be around 80MB for the full dataset
else:
    print("File not found. Downloading...")
    !wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    !unzip trainingandtestdata.zip

File exists. Size: 227.74 MB


In [ ]:
import pandas as pd

# File path
file_path = '/content/training.1600000.processed.noemoticon.csv'

# Define column names
columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

try:
    df = pd.read_csv(
        file_path,
        encoding='latin-1',
        names=columns,
        usecols=['target', 'text'],
        header=None,
        on_bad_lines='skip'  # ✅ Correct for pandas >=1.3.0
        # nrows=100000        # Optional: limit rows for testing
    )
    print(f"\n✅ Successfully loaded {len(df)} tweets.")
except Exception as e:
    print(f"\n❌ Failed to load data: {e}")




✅ Successfully loaded 1600000 tweets.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import re


In [ ]:
# Convert target labels safely
def safe_convert(val):
    try:
        return int(val)
    except:
        return None

# Apply conversion
df['label'] = df['target'].apply(safe_convert)
df = df.dropna(subset=['label'])  # Drop rows with invalid labels
df['label'] = df['label'].astype(int)

# Keep only binary labels: 0 (negative), 4 (positive)
df = df[df['label'].isin([0, 4])]
df['label'] = df['label'].map({0: 0, 4: 1})  # Map to binary

# Check distribution
print("✅ Final label distribution:")
print(df['label'].value_counts())


✅ Final label distribution:
label
0    800000
1    800000
Name: count, dtype: int64


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize once
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = str(text).lower()

    # Remove URLs, mentions, hashtags
    text = re.sub(r'http\S+|www\S+|@\w+|#\w+', '', text)

    # Remove punctuation, numbers, and special characters
    text = re.sub(r'[^a-z\s]', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize and lemmatize
    tokens = text.split()
    cleaned = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(cleaned)

# Apply to your dataframe
df['clean_text'] = df['text'].apply(preprocess)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'],  # input features
    df['label'],       # target labels
    test_size=0.2,     # 80/20 split
    random_state=42,   # reproducibility
    stratify=df['label']  # preserve class distribution
)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1, 2)  # Unigrams + Bigrams
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10]}
svm = LinearSVC(max_iter=10000)
grid = GridSearchCV(svm, param_grid, cv=3, scoring='accuracy')
grid.fit(X_train_tfidf, y_train)

print("Best SVM params:", grid.best_params_)
print("Best SVM accuracy:", grid.best_score_)

y_pred = grid.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))



Best SVM params: {'C': 0.1}
Best SVM accuracy: 0.767827344114707
              precision    recall  f1-score   support

           0       0.79      0.73      0.76    160000
           1       0.75      0.80      0.78    160000

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize the model
logreg = LogisticRegression(
    max_iter=200,       # Increase if convergence warnings appear
    C=1.0,              # Regularization strength (can tune this)
    solver='liblinear'  # Good for small datasets or L1 regularization
)

# Fit the model
logreg.fit(X_train_tfidf, y_train)

# Predict
y_pred = logreg.predict(X_test_tfidf)

# Evaluate
print("📊 Logistic Regression Performance:")
print(classification_report(y_test, y_pred))


📊 Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.79      0.74      0.76    160000
           1       0.75      0.80      0.78    160000

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Initialize and train the model
model = MultinomialNB(alpha=0.5)
model.fit(X_train_tfidf, y_train)

# Predict on test data
y_pred = model.predict(X_test_tfidf)

# Evaluate performance
print("\n📊 Classification Report (Multinomial Naive Bayes):")
print(classification_report(y_test, y_pred, target_names=['negative', 'positive']))



📊 Classification Report (Multinomial Naive Bayes):
              precision    recall  f1-score   support

    negative       0.75      0.75      0.75    160000
    positive       0.75      0.75      0.75    160000

    accuracy                           0.75    320000
   macro avg       0.75      0.75      0.75    320000
weighted avg       0.75      0.75      0.75    320000



In [ ]:
def predict_sentiment(text):
    try:
        # Step 1: Preprocess the input
        cleaned = preprocess(text)

        if not cleaned.strip():
            return {
                "text": text,
                "error": "Text too short or invalid after preprocessing"
            }

        # Step 2: Transform the cleaned text to TF-IDF features
        vectorized = tfidf.transform([cleaned])

        # Step 3: Make prediction
        prediction = model.predict(vectorized)[0]

        # Step 4: Predict probabilities (if supported by model)
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(vectorized)[0]
        else:
            proba = [None, None]

        # Step 5: Construct response
        label_map = {0: "negative", 1: "positive"}
        return {
            "text": text,
            "prediction": label_map.get(prediction, "unknown"),
            "confidence": {
                "negative": f"{(proba[0]*100):.1f}%" if proba[0] is not None else "N/A",
                "positive": f"{(proba[1]*100):.1f}%" if proba[1] is not None else "N/A"
            }
        }

    except Exception as e:
        return {"text": text, "error": str(e)}


In [ ]:
# === Test Cases ===
test_texts = [
    "I love this product! It's amazing.",
    "This is terrible and awful.",
    "",
    "123 456",
    "Very good",
    "I hate this so much!",
    "Not bad at all"
]

print("\n=== Sentiment Analysis Test Results ===")
for text in test_texts:
    result = predict_sentiment(text)
    print(f"\nText: '{text}'")
    print("Result:", result)



=== Sentiment Analysis Test Results ===

Text: 'I love this product! It's amazing.'
Result: {'text': "I love this product! It's amazing.", 'prediction': 'positive', 'confidence': {'negative': '17.3%', 'positive': '82.7%'}}

Text: 'This is terrible and awful.'
Result: {'text': 'This is terrible and awful.', 'prediction': 'negative', 'confidence': {'negative': '94.3%', 'positive': '5.7%'}}

Text: ''
Result: {'text': '', 'error': 'Text too short or invalid after preprocessing'}

Text: '123 456'
Result: {'text': '123 456', 'error': 'Text too short or invalid after preprocessing'}

Text: 'Very good'
Result: {'text': 'Very good', 'prediction': 'positive', 'confidence': {'negative': '29.6%', 'positive': '70.4%'}}

Text: 'I hate this so much!'
Result: {'text': 'I hate this so much!', 'prediction': 'negative', 'confidence': {'negative': '87.9%', 'positive': '12.1%'}}

Text: 'Not bad at all'
Result: {'text': 'Not bad at all', 'prediction': 'negative', 'confidence': {'negative': '79.4%', 'positi

In [ ]:
import joblib

# Save model and vectorizer
joblib.dump(model, 'naive_bayes_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']